In [ ]:
import os, sys
try:
    from synapse.lib.jupyter import *
    import synapse.common as s_common
    import synapse.lib.parser as s_parser
    import synapse.lib.output as s_output
    import synapse.tools.genpkg as s_t_genpkg
except ImportError as e:
    # Insert the root path of the repository to sys.path.
    # This assumes the notebook is located three directories away
    # From the root synapse directory. It may need to be varied
    synroot = os.path.abspath('../../../')
    sys.path.insert(0, synroot)
    from synapse.lib.jupyter import *
    import synapse.common as s_common
    import synapse.lib.parser as s_parser
    import synapse.lib.output as s_output
    import synapse.tools.genpkg as s_t_genpkg

In [ ]:
# Get a Cortex we can use for testing.
core = await getTempCoreCmdr()

# get the base dir for the demopkg
dpkg_fp = getDocPath('foopkg/foopkg.yml')

In [ ]:
# Ensure the yaml content is valid
_contents = s_common.yamlload(dpkg_fp)
assert _contents.get('name') == 'foopkg'
with s_common.genfile(dpkg_fp) as fd:
    buf = fd.read()
print(buf.decode())


In [ ]:
dpkg_foomod = getDocPath('foopkg/storm/modules/foomod')

with s_common.genfile(dpkg_foomod) as fd:
    buf = fd.read()
storm = buf.decode()
# Ensure the package is valid storm
assert len(storm) > 1
_ = s_parser.parseQuery(storm)
print(storm)

In [ ]:
dpkg_foocmd = getDocPath('foopkg/storm/commands/foocmd')
with s_common.genfile(dpkg_foocmd) as fd:
    buf = fd.read()
storm = buf.decode()
# Ensure the package is valid storm
assert len(storm) > 1
_ = s_parser.parseQuery(storm)
print(storm)

In [ ]:
# Load the package into the Cortex
lurl = core.core._core.getLocalUrl()
args = ['--push', lurl, dpkg_fp]
outp = s_output.OutPutStr()
retn = await s_t_genpkg.main(args, outp)
assert retn == 0
_ = await core.eval('syn:cmd:package=foopkg', num=1)
_ = await core.eval('[inet:ipv4=192.168.0.113 :asn=20]')
podes = await core.eval('inet:ipv4=192.168.0.113 | foocmd', cmdr=True)
assert podes[0][1]['props'].get('asn') == 40


In [ ]:
# Setup the directory we'll pave the storm package contents into
coredirn = core.core._core.dirn
save_fp = os.path.join(coredirn, 'save.json')
args = ['--save', save_fp, dpkg_fp]
outp = s_output.OutPutStr()
retn = await s_t_genpkg.main(args, outp)
assert retn == 0
data = s_common.jsload(save_fp)
assert data.get('name') == 'foopkg'
assert 'commands' in data
assert 'modules' in data

In [ ]:
await core.fini()